# Using the dysts database to investigate the connection between symbolic regression and invariant manifolds

In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.integrate import odeint
from sklearn.metrics import mean_squared_error
from dysts.base import make_trajectory_ensemble
from dysts.base import get_attractor_list
import dysts.flows as flows
import dysts.datasets as datasets
import time

from utils import *

import pysindy as ps

# Ignore warnings
# import warnings
# warnings.filterwarnings('ignore')

# Seed the random number generators for reproducibility
np.random.seed(100)

# Chaotic System Initialization

This experiment include 73 chaotic, polynomially nonlinear systems provided by the database from William Gilpin. "Chaos as an interpretable benchmark for forecasting and data-driven modelling" Advances in Neural Information Processing Systems (NeurIPS) 2021 https://arxiv.org/abs/2110.05266.

In [ ]:
t1 = time.time()

# system list of the polynomially nonlinear systems in the dysts database
# Something seems really off about the AtmosphericRegime, so omitting for now!
systems_list = ["Aizawa", "Bouali2", # "AtmosphericRegime", # Arneodo does not Lyapunov Spectrum calculated
                "GenesioTesi", "HyperBao", "HyperCai", "HyperJha", 
                "HyperLorenz", "HyperLu", "HyperPang", "Laser",
                "Lorenz", "LorenzBounded", "MooreSpiegel", "Rossler", "ShimizuMorioka",
                "HenonHeiles", "GuckenheimerHolmes", "Halvorsen", "KawczynskiStrizhak",
                "VallisElNino", "RabinovichFabrikant", "NoseHoover", "Dadras", "RikitakeDynamo",
                "NuclearQuadrupole", "PehlivanWei", "SprottTorus", "SprottJerk", "SprottA", "SprottB",
                "SprottC", "SprottD", "SprottE", "SprottF", "SprottG", "SprottH", "SprottI", "SprottJ",
                "SprottK", "SprottL", "SprottM", "SprottN", "SprottO", "SprottP", "SprottQ", "SprottR",
                "SprottS", "Rucklidge", "Sakarya", "RayleighBenard", "Finance", "LuChenCheng",
                "LuChen", "QiChen", "ZhouChen", "BurkeShaw", "Chen", "ChenLee", "WangSun", "DequanLi",
                "NewtonLiepnik", "HyperRossler", "HyperQi", "Qi", "LorenzStenflo", "HyperYangChen", 
                "HyperYan", "HyperXu", "HyperWang", "Hadley", "HindmarshRose",
               ]
alphabetical_sort = np.argsort(systems_list)
systems_list = np.array(systems_list)[alphabetical_sort]

# attributes list
attributes = [
    "maximum_lyapunov_estimated",
    "lyapunov_spectrum_estimated",
    "embedding_dimension",
    "parameters",
    "dt",
    "hamiltonian",
    "period",
    "unbounded_indices"
]

# Get attributes
all_properties = dict()
for i, equation_name in enumerate(systems_list):
    eq = getattr(flows, equation_name)()
    attr_vals = [getattr(eq, item, None) for item in attributes]
    all_properties[equation_name] = dict(zip(attributes, attr_vals))
    print(i, equation_name, all_properties[equation_name])
    
    
# Get training and testing trajectories for all the experimental systems 
all_sols_train, all_t_train, all_sols_test, all_t_test = load_data(
    systems_list, all_properties, 
    n=5000, pts_per_period=200,
    random_bump=False,
)
test_trajectories, test_trajectories_time = make_test_trajectories(
    systems_list,
    all_properties,
    n=2000,
    pts_per_period=20,
    random_bump=False,
    include_transients=False,
    approximate_center=0.0,  # approximate center of the attractor
    n_trajectories=10,
)
        
t2 = time.time()
print('Took ', t2 - t1, ' seconds to load the systems')

0 Aizawa {'maximum_lyapunov_estimated': 0.08947878317195473, 'lyapunov_spectrum_estimated': [0.08947878317195473, 0.020305496211675024, -0.3090729926541944], 'embedding_dimension': 3, 'parameters': {'a': 0.95, 'b': 0.7, 'c': 0.6, 'd': 3.5, 'e': 0.25, 'f': 0.1}, 'dt': 0.0009043, 'hamiltonian': False, 'period': 2.5837, 'unbounded_indices': []}
1 Bouali2 {'maximum_lyapunov_estimated': 0.02038922500462878, 'lyapunov_spectrum_estimated': [0.020389225004628773, 0.0033455417942589724, -0.009602768742817189], 'embedding_dimension': 3, 'parameters': {'a': 3.0, 'b': 2.2, 'bb': 0, 'c': 0, 'g': 1.0, 'm': -0.0026667, 'y0': 1.0}, 'dt': 0.0004124, 'hamiltonian': False, 'period': 2.6779, 'unbounded_indices': []}
2 BurkeShaw {'maximum_lyapunov_estimated': 2.3453555864112117, 'lyapunov_spectrum_estimated': [2.3453555864112117, 0.027056468824027846, -11.225316054517004], 'embedding_dimension': 3, 'parameters': {'e': 13, 'n': 10}, 'dt': 0.0001175, 'hamiltonian': False, 'period': 0.78333, 'unbounded_indice

30 LorenzStenflo {'maximum_lyapunov_estimated': 0.44944459925783387, 'lyapunov_spectrum_estimated': [0.44944459925783387, 0.026077542739630446, -2.7358399678801906, -3.4213273148340293], 'embedding_dimension': 4, 'parameters': {'a': 2, 'b': 0.7, 'c': 26, 'd': 1.5}, 'dt': 0.0006259, 'hamiltonian': False, 'period': 3.8877142857142855, 'unbounded_indices': []}
31 LuChen {'maximum_lyapunov_estimated': 0.6651845739025707, 'lyapunov_spectrum_estimated': [0.6651845739025707, 0.1450338199952451, -22.56999542505058], 'embedding_dimension': 3, 'parameters': {'a': 36, 'b': 3, 'c': 18}, 'dt': 0.0001216, 'hamiltonian': False, 'period': 2.3385, 'unbounded_indices': []}
32 LuChenCheng {'maximum_lyapunov_estimated': 0.35106210160478324, 'lyapunov_spectrum_estimated': [0.35106210160478324, -0.012414362698175424, -11.556516575635818], 'embedding_dimension': 3, 'parameters': {'a': -10, 'b': -4, 'c': 18.1}, 'dt': 0.0001108, 'hamiltonian': False, 'period': 0.92333, 'unbounded_indices': []}
33 MooreSpiegel 

62 SprottO {'maximum_lyapunov_estimated': 0.06685684927644238, 'lyapunov_spectrum_estimated': [0.06685684927644238, 0.011533284505120292, -0.3165440349833235], 'embedding_dimension': 3, 'parameters': {'a': 2.7}, 'dt': 0.001951, 'hamiltonian': False, 'period': 9.755, 'unbounded_indices': []}
63 SprottP {'maximum_lyapunov_estimated': 0.10261657056457861, 'lyapunov_spectrum_estimated': [0.1026165705645786, 0.00025136328640969387, -0.4843551814513015], 'embedding_dimension': 3, 'parameters': {'a': 2.7}, 'dt': 0.001549, 'hamiltonian': False, 'period': 5.3414, 'unbounded_indices': []}
64 SprottQ {'maximum_lyapunov_estimated': 0.13698250147166127, 'lyapunov_spectrum_estimated': [0.13698250147166127, 0.0010386716745898255, -0.6140516336902575], 'embedding_dimension': 3, 'parameters': {'a': 3.1, 'b': 0.5}, 'dt': 0.001618, 'hamiltonian': False, 'period': 4.8443, 'unbounded_indices': []}
65 SprottR {'maximum_lyapunov_estimated': 0.07608481335025485, 'lyapunov_spectrum_estimated': [0.0760848133502

61 SprottN(name='SprottN', params={}, random_state=None)
62 SprottO(name='SprottO', params={'a': 2.7}, random_state=None)
63 SprottP(name='SprottP', params={'a': 2.7}, random_state=None)
64 SprottQ(name='SprottQ', params={'a': 3.1, 'b': 0.5}, random_state=None)
65 SprottR(name='SprottR', params={'a': 0.9, 'b': 0.4}, random_state=None)
66 SprottS(name='SprottS', params={}, random_state=None)
67 SprottTorus(name='SprottTorus', params={}, random_state=None)
68 VallisElNino(name='VallisElNino', params={'b': 102, 'c': 3, 'p': 0}, random_state=None)
69 WangSun(name='WangSun', params={'a': 0.2, 'b': -0.01, 'd': -0.4, 'e': -1.0, 'f': -1.0, 'q': 1.0}, random_state=None)
70 ZhouChen(name='ZhouChen', params={'a': 2.97, 'b': 0.15, 'c': -3.0, 'd': 1, 'e': -8.78}, random_state=None)
0 Aizawa(name='Aizawa', params={'a': 0.95, 'b': 0.7, 'c': 0.6, 'd': 3.5, 'e': 0.25, 'f': 0.1}, random_state=None)
1 Bouali2(name='Bouali2', params={'a': 3.0, 'b': 2.2, 'bb': 0, 'c': 0, 'g': 1.0, 'm': -0.0026667, 'y0': 1.

# Calculate the true coefficients
Function from utils.py file reads in all the dysts database functions defining the systems of ODEs, and extracts all the coefficients.

In [ ]:
num_attractors = len(systems_list)

lyap_list = []
dimension_list = []
param_list = []
# Calculate scale separation
scale_list = []
linear_scale_list = []

for system in systems_list:
    lyap_list.append(all_properties[system]['maximum_lyapunov_estimated'])
    dimension_list.append(all_properties[system]['embedding_dimension'])
    param_list.append(all_properties[system]['parameters'])
    # Ratio of largest to smallest timescales
    # scale_list.append(all_properties[system]['period'] / all_properties[system]['dt'])


true_coefficients = make_dysts_true_coefficients(systems_list, 
                                                 all_sols_train, 
                                                 dimension_list, 
                                                 param_list)

# Want ratio of largest to smallest LINEAR or LYAPUNOV EXPONENT timescales 
for i, system in enumerate(systems_list):
    linear_coefs = true_coefficients[i][:, :dimension_list[i]] 
    max_linear = np.max(abs(linear_coefs))
    min_linear = np.min(abs(linear_coefs[linear_coefs != 0.0]))
    linear_scale_list.append(max_linear / min_linear)
    # print(all_properties[system]['lyapunov_spectrum_estimated'])
    sorted_spectrum = np.sort((np.array(all_properties[system]['lyapunov_spectrum_estimated'])))
    print(sorted_spectrum, np.sum(np.array(sorted_spectrum > 0.0, dtype=int)))
    lambda_max = sorted_spectrum[-1]
    lambda_min = sorted_spectrum[0]

    #print(i, system, all_properties[system]['lyapunov_spectrum_estimated'])
    scale_list.append(lambda_max / lambda_min)
    print(i, system, scale_list[i])


## Trajectory Visualization
Visualizing the training and testing trajectories helps us verify if the time series data is coming from the strange attractors or from transients in the evolution.

In [ ]:
t1 = time.time()

# Plot the training and testing trajectories for all the chaotic systems
num_cols = 5
num_rows = int(np.ceil(len(all_sols_train) / num_cols))
fig = plt.figure(figsize=(num_cols * 2, num_rows * 2))

gs = plt.matplotlib.gridspec.GridSpec(num_rows, num_cols)
gs.update(wspace=0.0, hspace=0.05) 

for i, attractor_name in enumerate(all_sols_train):
    
    x_train = all_sols_train[attractor_name]
    x_test = all_sols_test[attractor_name]
    t_train = all_t_train[attractor_name]
    t_test = all_t_test[attractor_name]
    
    plt.subplot(gs[i])
    plt.plot(x_train[:, 0], x_train[:, 1], 'k'
             , linewidth=0.25)
    plt.plot(x_test[:, 0], x_test[:, 1], 'r', linewidth=0.25)
    plt.title(attractor_name, y=-0.1, fontsize=14)
    plt.gca().axis('off')
            
# plt.savefig('polynomial_attractors.jpg')
# plt.savefig('polynomial_attractors.pdf')
t2 = time.time()
print('Took ', t2 - t1, ' seconds to plot the systems')

In [ ]:
t1 = time.time()

# Plot the training and testing trajectories for all the chaotic systems
num_cols = 1
num_rows = len(all_sols_train)
fig = plt.figure(figsize=(20, num_rows * 2))

gs = plt.matplotlib.gridspec.GridSpec(num_rows, num_cols)
gs.update(wspace=0.0, hspace=0.05) 

for i, attractor_name in enumerate(all_sols_train):
    
    x_train = all_sols_train[attractor_name]
    t_train = all_t_train[attractor_name]
    
    plt.subplot(gs[i])
    for j in range(dimension_list[i]):
        #plt.subplot(gs[i + j])
        plt.plot(x_train[:, j])
        plt.grid(True)
    plt.title(attractor_name, y=-0.1, fontsize=14)
    #plt.gca().axis('off')
    plt.legend(['x', 'y', 'z'])
            
# plt.savefig('polynomial_attractors.jpg')
# plt.savefig('polynomial_attractors.pdf')
t2 = time.time()
print('Took ', t2 - t1, ' seconds to plot the systems')

# Use the RMSE errors of $\dot{x}$ on a testing trajectory to guide a hyperparameter scan for the best threshold to use in the STLSQ algorithm. 
This uses a modified "Rudy algorithm" where the best model at each iteration is the one that minimizes the sum of the normalized RMSE error of $\dot{x}$ on a test trajectory and the number of nonzero terms * the l0 penalty value. Note that some models will fail to produce correct models (as measured against the true coefficients) even in the noiseless case, unless the data is well sampled. We will use the results of this scan to run ensembling SINDy, which will allow us to conclude about the effects of scale separation. 

In [ ]:
t1 = time.time()

# Note, defaults to Rudy Algorithm 2 using the x_dot RMSE error
# as the metric for success. Use coef_error_metric = True to use
# the normalized coefficient error as the metric for success
(xdot_rmse_errors, xdot_coef_errors, x_dot_tests, x_dot_test_preds,
predicted_coefficients, best_threshold_values, 
best_normalized_coef_errors, models, condition_numbers) = Pareto_scan(
    systems_list, dimension_list, true_coefficients,
    all_sols_train, all_t_train, all_sols_test, all_t_test, l0_penalty=1e-5, 
    normalize_columns=False,
    # coef_error_metric=True
)
            
t2 = time.time()
print('Total time to compute = ', t2 - t1, ' seconds')
print('Condition numbers = ', condition_numbers)


### Normalized Error
Below, we can plot the individual coefficient errors for every system (but this is a lot of information and plots!) and we can plot the total normalized coefficient errors, RMSE errors, and best thresholds for each system, along with a linear fit (on a semi-log plot, so really an exponential fit) on the coefficient error. The fit indicates that scale separation and the invariant manifolds matter!

In [ ]:
plot_individual_coef_errors(
    all_sols_train,
    predicted_coefficients,
    true_coefficients,
    dimension_list,
    systems_list,
    models
)

In [ ]:

# Plot the training and testing trajectories for all the chaotic systems
num_cols = 5
num_rows = int(np.ceil(len(all_sols_train) / num_cols))
fig = plt.figure(figsize=(num_cols * 2, num_rows * 2))

gs = plt.matplotlib.gridspec.GridSpec(num_rows, num_cols)
gs.update(wspace=0.0, hspace=0.05) 

for i, attractor_name in enumerate(all_sols_train):
    x_dot_test = x_dot_tests[i]
    x_dot_test_pred = x_dot_test_preds[i]
    plt.subplot(gs[i])
    plt.plot(x_dot_test[:, 0], x_dot_test[:, 1], 'k'
                 , linewidth=0.25)
    plt.plot(x_dot_test_pred[:, 0], x_dot_test_pred[:, 1], 'r', linewidth=0.25)
    plt.title(attractor_name, y=-0.1, fontsize=14)
    plt.gca().axis('off')
    

In [ ]:
plot_coef_errors(
    all_sols_train,
    best_normalized_coef_errors,
    xdot_rmse_errors,
    best_threshold_values,
    condition_numbers, #scale_list,
    systems_list,
    normalize_columns=False
)


## Okay, we have successfully found the Pareto optimal models for the given data. Now we build an ensemble of models with this "best threshold" and use the statistics of these models to conclude about the scale separation. 

In [ ]:
# Follow up by doing an ensembling fit using this optimal threshold
# ensemble_coefs = {}
# poly_library = ps.PolynomialLibrary(degree=4)
# for i, attractor_name in enumerate(systems_list):
#     print(i, " / ", num_attractors, ", System = ", attractor_name)
    
#     x_train = all_sols_train[attractor_name]
#     t_train = all_t_train[attractor_name]

#     optimizer = ps.STLSQ(resample
#         threshold=best_threshold_values[attractor_name][0],
#         alpha=1e-5,
#         max_iter=100,
#         normalize_columns=True,
#         ridge_kw={"tol": 1e-10},
#     )
#     if dimension_list[i] == 3:
#         input_names = ['x', 'y', 'z']
#     else:
#         input_names = ['x', 'y', 'z', 'w']
#     model = ps.SINDy(
#         feature_library=poly_library, 
#         optimizer=optimizer, 
#         feature_names=input_names,
#     )
#     model.fit(x_train, t=t_train, quiet=True, ensemble=True, n_models=10)
#     ensemble_coefs[attractor_name] = model.coef_list

In [ ]:
t1 = time.time()

x_pred, x_dot_pred, coef_lists = run_ensembling(
    systems_list,
    all_sols_train,
    all_t_train,
    test_trajectories, 
    test_trajectories_time,
    dimension_list,
    best_threshold_values,
    alpha=1e-5,
    optimizer_max_iter=100,
    normalize_columns=True,
    n_models=20,
)

t2 = time.time()
print('Took ', t2 - t1, ' seconds to ensemble the systems')

In [ ]:
x_pred['Aizawa'].shape

In [ ]:
num_trajectories = x_pred['Aizawa'].shape[2]
num_models = x_pred['Aizawa'].shape[1]
for i in range(num_models):
    for j in range(num_trajectories):
        plt.figure()
        ax = plt.axes(projection='3d')
        ax.plot(test_trajectories['Aizawa'][:, j, 0], test_trajectories['Aizawa'][:, j, 1], test_trajectories['Aizawa'][:, j, 2], 'k')
        ax.plot(x_pred['Aizawa'][:, i, j, 0], x_pred['Aizawa'][:, i, j, 1], x_pred['Aizawa'][:, i, j, 2], 'r')


In [ ]:
test_trajectories['Aizawa'].shape